In [24]:
import numpy as np
from scipy import stats
import pandas as pd

# 读取CSV文件
data = pd.read_csv('problem2.csv')

# 提取相关数据列（假设数据在第一行）
S = data['Underlying'].iloc[0]  # 标的资产价格
K = data['Strike'].iloc[0]  # 执行价格
T = data['TTM'].iloc[0] / 255  # 距到期时间（转换为年）
r = data['RF'].iloc[0]  # 无风险利率
option_price = data['CallPrice'].iloc[0]  # 期权市场价格

a. Calculate the implied volatility

In [25]:
import numpy as np
from scipy import stats

# Black-Scholes模型的Call期权定价函数
def black_scholes_call(S, K, T, r, sigma):
    if sigma == 0:  # 如果波动率为零，直接返回一个简单的期权价值（欧式期权）
        return max(0, S - K)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)
    return call_price

# 计算隐含波动率的函数（用牛顿法）
def implied_volatility(S, K, T, r, market_price, option_type='call'):
    def objective_function(sigma):
        if option_type == 'call':
            return black_scholes_call(S, K, T, r, sigma) - market_price
        else:
            return black_scholes_put(S, K, T, r, sigma) - market_price

    # 初始猜测值
    implied_vol = 0.2  
    tol = 1e-5  # 容忍误差
    max_iter = 100  # 最大迭代次数
    
    for _ in range(max_iter):
        f_val = objective_function(implied_vol)
        if abs(f_val) < tol:
            return implied_vol
        
        # 数值微分计算vega，避免出现除零错误
        epsilon = 1e-4  # 增大epsilon，避免除零错误
        vega = (black_scholes_call(S, K, T, r, implied_vol + epsilon) - black_scholes_call(S, K, T, r, implied_vol - epsilon)) / (2 * epsilon)
        
        # 防止vega接近零
        if vega == 0:
            break
        
        # 使用牛顿法更新隐含波动率
        implied_vol -= f_val / vega
    
    return implied_vol

# 计算隐含波动率
sigma = implied_volatility(S, K, T, r, market_price, option_type='call')
print(f"隐含波动率: {sigma}")


隐含波动率: 0.16344752534381815


b. Calculate the put price

In [26]:
# Black-Scholes模型的Put期权定价函数
def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)
    return put_price

# 计算看跌期权价格
put_price = black_scholes_put(S, K, T, r, sigma)
print(f"看跌期权价格: {put_price}")

看跌期权价格: 4.708963573436911


c. Calculate Call Delta

In [27]:
# 计算看涨期权Delta
d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
call_delta = stats.norm.cdf(d1)
print(f"看涨期权Delta: {call_delta}")


看涨期权Delta: 0.5345004518643305


d. Calculate Call Vega

In [28]:
# 计算看涨期权Vega
call_vega = S * np.sqrt(T) * stats.norm.pdf(d1)
print(f"看涨期权Vega: {call_vega}")


看涨期权Vega: 29.644706981606177


e. Calculate the Put Delta

In [29]:
# 计算看跌期权Delta
put_delta = stats.norm.cdf(d1) - 1
print(f"看跌期权Delta: {put_delta}")


看跌期权Delta: -0.4654995481356695


f. Calculate the Put Vega

In [30]:
# 计算看跌期权Vega
put_vega = S * np.sqrt(T) * stats.norm.pdf(d1)
print(f"看跌期权Vega: {put_vega}")


看跌期权Vega: 29.644706981606177


g. You observe a decrease in implied volatility by 5%. What is the profit or loss of your
position?

In [31]:
# 假设隐含波动率下降5%，计算损益
new_sigma = sigma * 0.95  # 隐含波动率下降5%
new_call_price = black_scholes_call(S, K, T, r, new_sigma)
new_put_price = black_scholes_put(S, K, T, r, new_sigma)

# 计算期权价格的变化（损益）
profit_or_loss = (new_call_price - option_price) + (new_put_price - put_price)
print(f"隐含波动率下降5%后的损益: {profit_or_loss}")


隐含波动率下降5%后的损益: -4.524574909647804


h. Describe the 2 methods you could have used to answer part g.

方法一：数值计算：
在这种方法中，我们通过计算隐含波动率的变化对期权价格的影响来得到损益。通过将隐含波动率减少5%，然后重新计算期权价格，最后得到价格变化（即损益）。
方法二：敏感度分析：
另一种方法是使用期权的敏感度指标（如Vega）。Vega衡量了隐含波动率变化对期权价格的影响。通过计算隐含波动率的变化并乘以Vega，可以得到期权价格的变化，从而推算出损益。